In [1]:
pip install streamlit-webrtc

  Using cached streamlit_webrtc-0.44.0-py3-none-any.whl (874 kB)
  Using cached aiortc-1.3.2-cp39-cp39-win_amd64.whl (992 kB)


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Lenovo\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\pip-install-iqgsqhi2\\netifaces_85ae06c0beb84d66b6696f3ca42ab8c9\\setup.py'"'"'; __file__='"'"'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\pip-install-iqgsqhi2\\netifaces_85ae06c0beb84d66b6696f3ca42ab8c9\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Lenovo\AppData\Local\Temp\pip-wheel-udvb_ogd'
       cwd: C:\Users\Lenovo\AppData\Local\Temp\pip-install-iqgsqhi2\netifaces_85ae06c0beb84d66b6696f3ca42ab8c9\
  Complete output (5 lines):
  running bdist_wheel
  running build
  running build_ext
  building 'netifaces' extension
  error: Micr


  Using cached pylibsrtp-0.7.1-cp39-cp39-win_amd64.whl (45 kB)
  Using cached pyee-9.0.4-py2.py3-none-any.whl (14 kB)
  Using cached google_crc32c-1.5.0-cp39-cp39-win_amd64.whl (27 kB)
  Using cached av-9.2.0-cp39-cp39-win_amd64.whl (24.4 MB)
  Using cached aioice-0.7.6-py3-none-any.whl (23 kB)
  Using cached netifaces-0.11.0.tar.gz (30 kB)
  Using cached dnspython-2.2.1-py3-none-any.whl (269 kB)
  Running setup.py clean for netifaces
Failed to build netifaces
    Running setup.py install for netifaces: started
    Running setup.py install for netifaces: finished with status 'error'


In [2]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mediapipe as mp 
import numpy as np 
import cv2 
 
cap = cv2.VideoCapture(0)

name = input("Enter the name of the data : ")

holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

X = []
data_size = 0

while True:
	lst = []

	_, frm = cap.read()

	frm = cv2.flip(frm, 1)

	res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))


	if res.face_landmarks:
		for i in res.face_landmarks.landmark:
			lst.append(i.x - res.face_landmarks.landmark[1].x)
			lst.append(i.y - res.face_landmarks.landmark[1].y)

		if res.left_hand_landmarks:
			for i in res.left_hand_landmarks.landmark:
				lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
				lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
		else:
			for i in range(42):
				lst.append(0.0)

		if res.right_hand_landmarks:
			for i in res.right_hand_landmarks.landmark:
				lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
				lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
		else:
			for i in range(42):
				lst.append(0.0)


		X.append(lst)
		data_size = data_size+1



	drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_CONTOURS)
	drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
	drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)

	cv2.putText(frm, str(data_size), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)

	cv2.imshow("window", frm)

	if cv2.waitKey(1) == 27 or data_size>99:
		cv2.destroyAllWindows()
		cap.release()
		break


np.save(f"{name}.npy", np.array(X))
print(np.array(X).shape)

Enter the name of the data : gg
(100, 1020)


In [5]:
import os  
import numpy as np 
import cv2 
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras.layers import Input, Dense 
from keras.models import Model
 
is_init = False
size = -1

label = []
dictionary = {}
c = 0

for i in os.listdir():
	if i.split(".")[-1] == "npy" and not(i.split(".")[0] == "labels"):  
		if not(is_init):
			is_init = True 
			X = np.load(i)
			size = X.shape[0]
			y = np.array([i.split('.')[0]]*size).reshape(-1,1)
		else:
			X = np.concatenate((X, np.load(i)))
			y = np.concatenate((y, np.array([i.split('.')[0]]*size).reshape(-1,1)))

		label.append(i.split('.')[0])
		dictionary[i.split('.')[0]] = c  
		c = c+1


for i in range(y.shape[0]):
	y[i, 0] = dictionary[y[i, 0]]
y = np.array(y, dtype="int32")

###  hello = 0 nope = 1 ---> [1,0] ... [0,1]

y = to_categorical(y)

X_new = X.copy()
y_new = y.copy()
counter = 0 

cnt = np.arange(X.shape[0])
np.random.shuffle(cnt)

for i in cnt: 
	X_new[counter] = X[i]
	y_new[counter] = y[i]
	counter = counter + 1



ip = Input(shape=(X.shape[1]))

m = Dense(512, activation="relu")(ip)
m = Dense(256, activation="relu")(m)

op = Dense(y.shape[1], activation="softmax")(m) 

model = Model(inputs=ip, outputs=op)

model.compile(optimizer='rmsprop', loss="categorical_crossentropy", metrics=['acc'])

model.fit((X), y, epochs=50)


model.save("model.h5")
np.save("labels.npy", np.array(label))

Epoch 1/50
25/25 [==============================] - 1s 12ms/step - loss: 1.7775 - acc: 0.3150
Epoch 2/50
25/25 [==============================] - 0s 12ms/step - loss: 1.3775 - acc: 0.5325
Epoch 3/50
25/25 [==============================] - 0s 15ms/step - loss: 1.1303 - acc: 0.6463
Epoch 4/50
25/25 [==============================] - 0s 14ms/step - loss: 0.9637 - acc: 0.6900
Epoch 5/50
25/25 [==============================] - 0s 11ms/step - loss: 0.8610 - acc: 0.7475
Epoch 6/50
25/25 [==============================] - 0s 11ms/step - loss: 0.7731 - acc: 0.7487
Epoch 7/50
25/25 [==============================] - 0s 10ms/step - loss: 0.7478 - acc: 0.7350
Epoch 8/50
25/25 [==============================] - 0s 10ms/step - loss: 0.6756 - acc: 0.7675
Epoch 9/50
25/25 [==============================] - 0s 12ms/step - loss: 0.6203 - acc: 0.7900
Epoch 10/50
25/25 [==============================] - 0s 12ms/step - loss: 0.6093 - acc: 0.7725
Epoch 11/50
25/25 [==============================] - 0s 12m

In [6]:
import cv2 
import numpy as np 
import mediapipe as mp 
from keras.models import load_model 


model  = load_model("model.h5")
label = np.load("labels.npy")



holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)



while True:
	lst = []

	_, frm = cap.read()

	frm = cv2.flip(frm, 1)

	res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))


	if res.face_landmarks:
		for i in res.face_landmarks.landmark:
			lst.append(i.x - res.face_landmarks.landmark[1].x)
			lst.append(i.y - res.face_landmarks.landmark[1].y)

		if res.left_hand_landmarks:
			for i in res.left_hand_landmarks.landmark:
				lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
				lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
		else:
			for i in range(42):
				lst.append(0.0)

		if res.right_hand_landmarks:
			for i in res.right_hand_landmarks.landmark:
				lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
				lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
		else:
			for i in range(42):
				lst.append(0.0)

		lst = np.array(lst).reshape(1,-1)

		pred = label[np.argmax(model.predict(lst))]

		print(pred)
		cv2.putText(frm, pred, (50,50),cv2.FONT_ITALIC, 1, (255,0,0),2)

		
	drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_CONTOURS)
	drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
	drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)

	cv2.imshow("window", frm)

	if cv2.waitKey(1) == 27:
		cv2.destroyAllWindows()
		cap.release()
		break

1/1 [==============================] - 0s 93ms/step
gg
1/1 [==============================] - 0s 19ms/step
gg
1/1 [==============================] - 0s 30ms/step
gg
1/1 [==============================] - 0s 24ms/step
gg
1/1 [==============================] - 0s 34ms/step
gg
1/1 [==============================] - 0s 22ms/step
gg
1/1 [==============================] - 0s 26ms/step
gg
1/1 [==============================] - 0s 18ms/step
gg
1/1 [==============================] - 0s 19ms/step
gg
1/1 [==============================] - 0s 17ms/step
gg
1/1 [==============================] - 0s 17ms/step
gg
1/1 [==============================] - 0s 27ms/step
gg
1/1 [==============================] - 0s 23ms/step
gg
1/1 [==============================] - 0s 22ms/step
gg
1/1 [==============================] - 0s 15ms/step
gg
1/1 [==============================] - 0s 22ms/step
gg
1/1 [==============================] - 0s 19ms/step
gg
1/1 [==============================] - 0s 32ms/step
gg
1/1 [=====

In [7]:
import streamlit as st
from streamlit_webrtc import webrtc_streamer
import av
import cv2 
import numpy as np 
import mediapipe as mp 
from keras.models import load_model
import webbrowser

model  = load_model("model.h5")
label = np.load("labels.npy")
holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

st.header("Emotion Based Music Recommender")

if "run" not in st.session_state:
	st.session_state["run"] = "true"

try:
	emotion = np.load("emotion.npy")[0]
except:
	emotion=""

if not(emotion):
	st.session_state["run"] = "true"
else:
	st.session_state["run"] = "false"

class EmotionProcessor:
	def recv(self, frame):
		frm = frame.to_ndarray(format="bgr24")

		##############################
		frm = cv2.flip(frm, 1)

		res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

		lst = []

		if res.face_landmarks:
			for i in res.face_landmarks.landmark:
				lst.append(i.x - res.face_landmarks.landmark[1].x)
				lst.append(i.y - res.face_landmarks.landmark[1].y)

			if res.left_hand_landmarks:
				for i in res.left_hand_landmarks.landmark:
					lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
					lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
			else:
				for i in range(42):
					lst.append(0.0)

			if res.right_hand_landmarks:
				for i in res.right_hand_landmarks.landmark:
					lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
					lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
			else:
				for i in range(42):
					lst.append(0.0)

			lst = np.array(lst).reshape(1,-1)

			pred = label[np.argmax(model.predict(lst))]

			print(pred)
			cv2.putText(frm, pred, (50,50),cv2.FONT_ITALIC, 1, (255,0,0),2)

			np.save("emotion.npy", np.array([pred]))

			
		drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_TESSELATION,
								landmark_drawing_spec=drawing.DrawingSpec(color=(0,0,255), thickness=-1, circle_radius=1),
								connection_drawing_spec=drawing.DrawingSpec(thickness=1))
		drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
		drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)


		##############################

		return av.VideoFrame.from_ndarray(frm, format="bgr24")

lang = st.text_input("Language")
singer = st.text_input("singer")

if lang and singer and st.session_state["run"] != "false":
	webrtc_streamer(key="key", desired_playing_state=True,
				video_processor_factory=EmotionProcessor)

btn = st.button("Recommend me songs")

if btn:
	if not(emotion):
		st.warning("Please let me capture your emotion first")
		st.session_state["run"] = "true"
	else:
		webbrowser.open(f"https://www.youtube.com/results?search_query={lang}+{emotion}+song+{singer}")
		np.save("emotion.npy", np.array([""]))
		st.session_state["run"] = "false"

ModuleNotFoundError: No module named 'streamlit_webrtc'

In [8]:
pip install streamlit-webrtc

  Using cached streamlit_webrtc-0.44.0-py3-none-any.whl (874 kB)
  Using cached aiortc-1.3.2-cp39-cp39-win_amd64.whl (992 kB)
  Using cached google_crc32c-1.5.0-cp39-cp39-win_amd64.whl (27 kB)
  Using cached av-9.2.0-cp39-cp39-win_amd64.whl (24.4 MB)
  Using cached pyee-9.0.4-py2.py3-none-any.whl (14 kB)
  Using cached pylibsrtp-0.7.1-cp39-cp39-win_amd64.whl (45 kB)
  Using cached aioice-0.7.6-py3-none-any.whl (23 kB)
  Using cached dnspython-2.2.1-py3-none-any.whl (269 kB)
  Using cached netifaces-0.11.0.tar.gz (30 kB)
  Running setup.py clean for netifaces
Failed to build netifaces
    Running setup.py install for netifaces: started
    Running setup.py install for netifaces: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Lenovo\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\pip-install-1co7oa93\\netifaces_ae112afa7776484a83e65a0f40b5c348\\setup.py'"'"'; __file__='"'"'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\pip-install-1co7oa93\\netifaces_ae112afa7776484a83e65a0f40b5c348\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Lenovo\AppData\Local\Temp\pip-wheel-7q2hn7_o'
       cwd: C:\Users\Lenovo\AppData\Local\Temp\pip-install-1co7oa93\netifaces_ae112afa7776484a83e65a0f40b5c348\
  Complete output (5 lines):
  running bdist_wheel
  running build
  running build_ext
  building 'netifaces' extension
  error: Micr